In [6]:
#imports
import random
import os
import json
import librosa
from tqdm import tqdm

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from model import FastSpeech2
from nemo.collections.tts.models import HifiGanModel
import soundfile as sf
import numpy as np

NOTE! Installing ujson may make loading annotations faster.


[NeMo W 2023-11-26 03:26:01 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-26 03:26:02 experimental:27] Module <class 'nemo.collections.tts.models.fastpitch_ssl.FastPitchModel_SSL'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-26 03:26:02 experimental:27] Module <class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-26 03:26:02 experimental:27] Module <class 'nemo.collections.tts.models.radtts.RadTTSModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-26 03:26:02 experimental:27] Module <class 'nemo.collections.tts.models.ssl_tts.SSLDisentangler'> is expe

In [2]:
import yaml
model_config = yaml.load(open('/workspace/nemo/vol/FastSpeech2/config/Nova/model.yaml',
                              "r"), Loader=yaml.FullLoader)
preprocess_config = yaml.load(open('/workspace/nemo/vol/FastSpeech2/config/Nova/preprocess.yaml',
                              "r"), Loader=yaml.FullLoader)

In [3]:
fp = FastSpeech2(model_config=model_config, preprocess_config=preprocess_config)
ckpt_file_path = '/workspace/nemo/vol/FastSpeech2/output/ckpt/Nova/50000.pth.tar'
checkpoint = torch.load(ckpt_file_path, map_location=torch.device('cuda'))
fp.load_state_dict(checkpoint['model'])
fp = fp.to('cuda')

In [4]:
# from torch import nn
# fp = nn.DataParallel(fp)

In [5]:
# checkpoint['model']['emotion_emb.weight'].shape

In [6]:
#helper functions
def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

In [7]:
# import pandas as pd
# df = pd.read_csv('/workspace/nemo/vol/LBM_EB/preprocessed_data/LBM_EB/paths.csv')

In [8]:
#rec = []

# speakers_path = "/workspace/nemo/vol/LBM_EB/preprocessed_data/LBM_EB/speakers.json"
# emotions_path = "/workspace/nemo/vol/LBM_EB/preprocessed_data/LBM_EB/emotions.json"

# with open(speakers_path, "r") as f:
#     speakers = json.load(f)
    
# with open(emotions_path, "r") as f:
#     emotions = json.load(f)

# for i, row in tqdm(df[886248:].iterrows()):
#     text_path = row.text_paths
#     wav_path = row.wav_paths
#     try:
#         with open(text_path, 'r') as f:
#             text = f.read()
#         duration = librosa.get_duration(filename=wav_path)
#         speaker = text_path.split('/')[-3]
#         emotion = text_path.split('/')[-2]
#         speaker_ = [v for k,v in speakers.items() if k==speaker][0]
#         emotion_ = [v for k,v in emotions.items() if k==emotion][0] 

#         r = {
#            "audio_filepath" : wav_path,
#            "text" : text,
#            "speaker": speaker_,
#            "emotion": emotion_,
#            "duration" : round(duration, 1),
#            "text_no_preprocessing" : text
#         }
#         rec.append(r)
#     except:
#         continue
    
    

In [9]:
#len(rec)

In [10]:
# import json

# # Sample array
# #data = [1, 2, 3, 4, 5]

# # Serialize the array to a JSON-formatted string
# json_data = json.dumps(rec)

# # Write the JSON string to a file
# with open('data_rec.json', 'w') as file:
#     file.write(json_data)

In [11]:
# import json
# with open('data_rec.json', 'r') as file:
#     rec = json.load(file)

In [12]:
# len(rec)

In [13]:
#data_json[0]

In [14]:
# file_paths = list()
rec = []

voices_path = '/workspace/nemo/vol/extvol3/NovaEmo/RawData/'
base_path = voices_path

preprocess_dir = "/workspace/nemo/vol/extvol3/NovaEmo/preprocessed_data/"
speakers_path = "/workspace/nemo/vol/extvol3/NovaEmo/preprocessed_data/speakers.json"
emotions_path = "/workspace/nemo/vol/extvol3/NovaEmo/preprocessed_data/emotions.json"

with open(preprocess_dir+'train.txt', 'r') as f:
    for line in tqdm(f):
        file, speaker, emotion, ph, text = line.split('|')
        if not speaker == 'NovaConvai':
            continue
        audio_filepath = os.path.join(base_path, speaker, emotion, '.'.join([file, 'wav']))
        text_filepath = os.path.join(base_path, speaker, emotion, '.'.join([file, 'lab']))
#         file_paths.append((audio_filepath, text_filepath))
        duration = librosa.get_duration(filename=audio_filepath)
        with open(text_filepath, 'r') as f:
            text = f.read()
        with open(speakers_path, "r") as f:
            speakers = json.load(f)
            speaker_ = [v for k,v in speakers.items() if k==speaker][0]
        with open(emotions_path, "r") as f:
            emotions = json.load(f)
            emotion_ = [v for k,v in emotions.items() if k==emotion][0]   
        r = {
           "audio_filepath" : audio_filepath,
           "text" : text,
           "speaker": speaker_,
           "emotion": emotion_,
           "duration" : round(duration,1),
           "text_no_preprocessing" : text
        }
        rec.append(r)
        
with open(preprocess_dir+'val.txt', 'r') as f:
    for line in f:
        file, speaker, emotion, ph, text = line.split('|')
        if not speaker == 'NovaConvai':
            continue
        audio_filepath = os.path.join(base_path, speaker, emotion, '.'.join([file, 'wav']))
        text_filepath = os.path.join(base_path, speaker, emotion, '.'.join([file, 'lab']))
#         file_paths.append((audio_filepath, text_filepath))
        duration = librosa.get_duration(filename=audio_filepath)
        with open(text_filepath, 'r') as f:
            text = f.read()
        with open(speakers_path, "r") as f:
            speakers = json.load(f)
            speaker_ = [v for k,v in speakers.items() if k==speaker][0]
        with open(emotions_path, "r") as f:
            emotions = json.load(f)
            emotion_ = [v for k,v in emotions.items() if k==emotion][0]   
        r = {
           "audio_filepath" : audio_filepath,
           "text" : text,
           "speaker": speaker_,
           "emotion": emotion_,
           "duration" : round(duration,1),
           "text_no_preprocessing" : text
        }
        rec.append(r)

14817it [00:04, 2973.61it/s]


In [15]:
# voices_path = '/workspace/nemo/vol/extvol3/NovaEmo/RawData/'

# base_path = voices_path

# speakers_path = "/workspace/nemo/vol/extvol3/NovaEmo/preprocessed_data/speakers.json"
# emotions_path = "/workspace/nemo/vol/extvol3/NovaEmo/preprocessed_data/emotions.json"

# rec = []

# # speaker = 'Female-V2'
# covered= []

# for speaker in os.listdir(base_path)

#     for emotion in os.listdir(base_path):
#         emotion_path = os.path.join(base_path, emotion)
#         print(emotion_path)

#         for file in os.listdir(emotion_path):
#             try:
#                 filename = file.split('.')[0]
#                 if filename in covered:
#                     continue
#                 covered.append(filename)

#                 audio_filepath = os.path.join(emotion_path, '.'.join([filename, 'wav'])) 
#                 text_filepath = os.path.join(emotion_path, '.'.join([filename, 'lab']))
#                 #print(text_filepath)
#                 duration = librosa.get_duration(filename=audio_filepath)
#                 with open(text_filepath, 'r') as f:
#                     text = f.read()
#                 with open(speakers_path, "r") as f:
#                     speakers = json.load(f)
#                     speaker_ = [v for k,v in speakers.items() if k==speaker][0]
#                 with open(emotions_path, "r") as f:
#                     emotions = json.load(f)
#                     emotion_ = [v for k,v in emotions.items() if k==emotion][0]   
#                 r = {
#                    "audio_filepath" : audio_filepath,
#                    "text" : text,
#                    "speaker": speaker_,
#                    "emotion": emotion_,
#                    "duration" : round(duration,1),
#                    "text_no_preprocessing" : text
#                 }
#                 rec.append(r)
#             except:
#                 continue
            


In [16]:
random.shuffle(rec)
train_len = int(0.9*len(rec))
train_rec = rec[:train_len]
val_rec = rec[train_len:]

In [17]:
train_manifest = f'./fastpitch_train.json'
with open(train_manifest, "w") as f:
    for s in train_rec:
        f.write(json.dumps(s) + '\n')
        
val_manifest = f'./fastpitch_val.json'
with open(val_manifest, "w") as f:
    for s in val_rec:
        f.write(json.dumps(s) + '\n')


In [18]:
#train_rec[0]

In [19]:
train_rec[0]

{'audio_filepath': '/workspace/nemo/vol/extvol3/NovaEmo/RawData/NovaConvai/NovaNeutral/0c710b31981437a90f6f02bfb9742e1053dfb61dc0d4b6d8f25c5005554d51eb_chunk_189.wav',
 'text': "That ain't all the Japanese talk.",
 'speaker': 163,
 'emotion': 2,
 'duration': 1.5,
 'text_no_preprocessing': "That ain't all the Japanese talk."}

In [20]:
from g2p_en import G2p
from string import punctuation
import re
from text import text_to_sequence


def read_lexicon(lex_path):
    lexicon = {}
    with open(lex_path) as f:
        for line in f:
            temp = re.split(r"\s+", line.strip("\n"))
            word = temp[0]
            phones = temp[1:]
            if word.lower() not in lexicon:
                lexicon[word.lower()] = phones
    return lexicon

g2p = G2p()
lexicon = read_lexicon(preprocess_config["path"]["lexicon_path"])

def preprocess_english(text, preprocess_config, g2p = g2p, lexicon = lexicon):
#     pdb.set_trace()
    text = text.rstrip(punctuation)    
    phones = []
    words = re.split(r"([,;.\-\?\!\s+])", text)
    for w in words:
        if w.lower() in lexicon:
            phones += lexicon[w.lower()]
        else:
            phones += list(filter(lambda p: p != " ", g2p(w)))
    phones = "{" + "}{".join(phones) + "}"
    phones = re.sub(r"\{[^\w\s]?\}", "{spn}", phones)
    phones = phones.replace("}{", " ")

#     print("Raw Text Sequence: {}".format(text))
#     print("Phoneme Sequence: {}".format(phones))
    sequence = np.array(
        text_to_sequence(
            phones, preprocess_config["preprocessing"]["text"]["text_cleaners"]
        )
    )

    return np.array(sequence)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# fp.to('cuda')
# fp.eval()

def create_hifigan_finetune_data(records, run= 'train', device='cuda'):    
    save_dir = Path(f"/workspace/nemo/vol/FastSpeech2/hifigan_mels/{run}")
    save_dir.mkdir(exist_ok=True, parents=True)
    
    fp.to(device)

    # Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
    
    for i, r in tqdm(enumerate(records)): 
        with torch.no_grad():
            text = r['text']
            speaker = r['speaker']
            emotion = r['emotion']
            ids = raw_texts = [text]
            speaker = torch.tensor([speaker]).to(device)
            emotion = torch.tensor([emotion]).to(device) 
            texts = torch.tensor([preprocess_english(text, preprocess_config)]).to(device)
            text_lens = torch.tensor([len(texts[0])]).to(device)
            max_len = max(text_lens)
            batchs = [(speaker, emotion, texts, text_lens, max_len )]
            #print(batchs[0])
            predictions = fp(*(batchs[0]))
            spectrogram = predictions[1].transpose(1, 2)

            save_path = save_dir / f"mel_{i}.npy"
            np.save(save_path, spectrogram[0].to('cpu').numpy())
            r["mel_filepath"] = str(save_path)

    hifigan_manifest_path = f"./hifigan_{run}_ft.json"
    with open(hifigan_manifest_path, "w") as f:
        for r in records:
            f.write(json.dumps(r) + '\n')

In [21]:
len(rec)

11903

In [22]:
# from synthesize import preprocess_english

# #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# fp.to('cuda')
# fp.eval()

# def create_hifigan_finetune_data(records, run= 'train', device='cuda'):    
#     beta_binomial_interpolator = BetaBinomialInterpolator()

#     save_dir = Path(f"/workspace/nemo/vol/FastSpeech2/hifigan_mels/{run}")
#     save_dir.mkdir(exist_ok=True, parents=True)
    
#     fp.to(device)

#     # Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
#     for i, r in tqdm(enumerate(records)):
#         audio = load_wav(r["audio_filepath"])
#         audio = torch.from_numpy(audio).unsqueeze(0).to(device)
#         audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)


#         with torch.no_grad():
#             text = r['text']
#             speaker = r['speaker']
#             emotion = r['emotion']
#             ids = raw_texts = [text[:100]]
#             speaker = torch.tensor([speaker]).to(device)
#             emotion = torch.tensor([emotion]).to(device) 
#             texts = torch.tensor([preprocess_english(text, preprocess_config)]).to(device)
#             text_lens = torch.tensor([len(texts[0])]).to(device)
#             batchs = [(ids, raw_texts, speaker, emotion, texts, text_lens, max(text_lens))]
#             #print(batchs[0][2:])
#             predictions = fp(*(batchs[0][2:]))
#             spectrogram = predictions[1].transpose(1, 2)

#     #         if "normalized_text" in r:
#     #             text = spec_model.parse(r["normalized_text"], normalize=False)
#     #         else:
#     #             text = spec_model.parse(r['text'])

#     #         text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)

#     #         spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

#     #         # Generate attention prior and spectrogram inputs for HiFi-GAN
#     #         attn_prior = torch.from_numpy(
#     #           beta_binomial_interpolator(spect_len.item(), text_len.item())
#     #         ).unsqueeze(0).to(text.device)

#     #         spectrogram = spec_model.forward(
#     #           text=text, 
#     #           input_lens=text_len, 
#     #           spec=spect, 
#     #           mel_lens=spect_len, 
#     #           attn_prior=attn_prior,
#     #         )[0]

#             save_path = save_dir / f"mel_{i}.npy"
#             np.save(save_path, spectrogram[0].to('cpu').numpy())
#             r["mel_filepath"] = str(save_path)

#     hifigan_manifest_path = f"./hifigan_{run}_ft.json"
#     with open(hifigan_manifest_path, "w") as f:
#         for r in records:
#             f.write(json.dumps(r) + '\n')

In [23]:
create_hifigan_finetune_data(train_rec, 'train')

0it [00:00, ?it/s][NeMo W 2023-11-25 11:53:29 nemo_logging:349] /tmp/ipykernel_857988/1772628250.py:66: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
      texts = torch.tensor([preprocess_english(text, preprocess_config)]).to(device)
    
10712it [05:17, 33.75it/s]


In [24]:
create_hifigan_finetune_data(val_rec, 'val')

1191it [00:35, 33.69it/s]


In [25]:
! cd hifigan_conf && unzip hifigan.zip
! cd hifigan_conf && cd hifigan && wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/hifigan/hifigan.yaml && cd .. 

/bin/bash: line 0: cd: hifigan_conf: No such file or directory
/bin/bash: line 0: cd: hifigan_conf: No such file or directory


In [26]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_hifigan.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_hifigan.nemo").write_bytes(nemo_files[0].read_bytes())

/root
Copying /root/.cache/torch/NeMo/NeMo_1.19.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo to ./


315386678

In [12]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/univnet.nemo")]
print(f"Copying {nemo_files[0]} to ./")
#Path("./tts_hifigan.nemo").write_bytes(nemo_files[0].read_bytes())

/root


IndexError: list index out of range

In [13]:
nemo_files

[]

In [27]:
#!pip install wandb -qU
#import wandb
#wandb.login()

In [28]:
wandb_api_key = "108bad0089a140932fbbe1c9e2ae182a1a228ffe"
#wandb.login()

In [29]:
!wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/hifigan_finetune.py

--2023-11-25 11:59:55--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/hifigan_finetune.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1192 (1.2K) [text/plain]
Saving to: ‘hifigan_finetune.py.1’

hifigan_finetune.py 100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-11-25 11:59:55 (80.7 MB/s) - ‘hifigan_finetune.py.1’ saved [1192/1192]



In [ ]:
!(HYDRA_FULL_ERROR=1 python hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=16 \
model.max_steps=100000 \
model.optim.lr=1e-4 \
~model.optim.sched \
train_dataset=hifigan_train_ft.json \
validation_datasets=hifigan_val_ft.json \
exp_manager.exp_dir=hifigan_ft \
+init_from_pretrained_model=nvidia/tts_hifigan \
trainer.check_val_every_n_epoch=5 \
trainer.log_every_n_steps=3 \
exp_manager.create_wandb_logger=true \
exp_manager.wandb_logger_kwargs.name='Nova_Emo_HG' \
exp_manager.wandb_logger_kwargs.project="TTS_convai"  \
)
# exp_manager.create_wandb_logger=true \
# exp_manager.wandb_logger_kwargs.name='Josh_emotional_HG' \
# exp_manager.wandb_logger_kwargs.project="TTS_convai"  \
# model/train_ds=train_ds_finetune \
# model/validation_ds=val_ds_finetune \

NOTE! Installing ujson may make loading annotations faster.
[NeMo W 2023-11-23 20:33:51 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-23 20:33:51 experimental:27] Module <class 'nemo.collections.tts.models.fastpitch_ssl.FastPitchModel_SSL'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-23 20:33:51 experimental:27] Module <class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-23 20:33:51 experimental:27] Module <class 'nemo.collections.tts.models.radtts.RadTTSModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
2023-11-23 20:33:52.604229: I tensorflow/core/platform/cpu_fe

In [42]:
from nemo.collections.tts.torch.data import VocoderDataset

In [1]:
print("""HYDRA_FULL_ERROR=1 python hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=32 \
model.max_steps=100000 \
model.optim.lr=1e-4 \
~model.optim.sched \
train_dataset=hifigan_train_ft.json \
validation_datasets=hifigan_val_ft.json \
exp_manager.exp_dir=hifigan_ft \
+init_from_pretrained_model=nvidia/tts_hifigan \
trainer.check_val_every_n_epoch=5 \
trainer.log_every_n_steps=3 \
exp_manager.create_wandb_logger=true \
exp_manager.wandb_logger_kwargs.name='Nova_Emo_HG' \
exp_manager.wandb_logger_kwargs.project="TTS_convai"  \
""")

HYDRA_FULL_ERROR=1 python hifigan_finetune.py --config-name=hifigan.yaml model.train_ds.dataloader_params.batch_size=32 model.max_steps=100000 model.optim.lr=1e-4 ~model.optim.sched train_dataset=hifigan_train_ft.json validation_datasets=hifigan_val_ft.json exp_manager.exp_dir=hifigan_ft +init_from_pretrained_model=nvidia/tts_hifigan trainer.check_val_every_n_epoch=5 trainer.log_every_n_steps=3 exp_manager.create_wandb_logger=true exp_manager.wandb_logger_kwargs.name='Nova_Emo_HG' exp_manager.wandb_logger_kwargs.project="TTS_convai"  


In [3]:
print("""HYDRA_FULL_ERROR=1 python hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=32 \
model.max_steps=100000 \
model.optim.lr=1e-4 \
~model.optim.sched \
train_dataset=hifigan_train_ft.json \
validation_datasets=hifigan_val_ft.json \
exp_manager.exp_dir=hifigan_ft \
+init_from_ptl_ckpt=/workspace/nemo/vol/FastSpeech2/hifiga02781n.ckpt \
trainer.check_val_every_n_epoch=5 \
trainer.log_every_n_steps=3 \
exp_manager.create_wandb_logger=true \
exp_manager.wandb_logger_kwargs.name='Nova_Emo_HG' \
exp_manager.wandb_logger_kwargs.project="TTS_convai"  \
""")

HYDRA_FULL_ERROR=1 python hifigan_finetune.py --config-name=hifigan.yaml model.train_ds.dataloader_params.batch_size=32 model.max_steps=100000 model.optim.lr=1e-4 ~model.optim.sched train_dataset=hifigan_train_ft.json validation_datasets=hifigan_val_ft.json exp_manager.exp_dir=hifigan_ft +init_from_ptl_ckpt=/workspace/nemo/vol/FastSpeech2/hifiga02781n.ckpt trainer.check_val_every_n_epoch=5 trainer.log_every_n_steps=3 exp_manager.create_wandb_logger=true exp_manager.wandb_logger_kwargs.name='Nova_Emo_HG' exp_manager.wandb_logger_kwargs.project="TTS_convai"  
